In [3]:
# ==============================================================
# 0. LIBRERÍAS Y CONFIGURACIÓN INICIAL
# ==============================================================

import os
import random
import math

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras

# --------------------------------------------------------------
# Fijamos semillas para reproducibilidad básica
# --------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# ==============================================================
# 1. CARGA DEL CONJUNTO DE DATOS LIMPIO
# ==============================================================

# Ruta al archivo limpio (ajusta el nombre si es necesario)
DATA_PATH = "listings_model_no_outliers.csv"

# Leemos el CSV en un DataFrame
df = pd.read_csv(DATA_PATH)

# Mostramos información básica para verificar que cargó bien
print("Tamaño dataframe:", df.shape)
print("Primeras columnas:", df.columns.tolist()[:20])

Tamaño dataframe: (40550, 147)
Primeras columnas: ['id', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'reviews_per_month', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'neighbourhood_cleansed_Barnet']
